General import statements. REMEMBER, DEEPLIFT_DIR needs to point to the deeplift directory WITHIN the deeplift repo

In [1]:
%matplotlib inline
from __future__ import division;
from __future__ import print_function;
from __future__ import absolute_import;
import sys, os;
from collections import OrderedDict, namedtuple;
import numpy as np;

#import deepLIFT stuff
scriptsDir = os.environ.get("DEEPLIFT_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable DEEPLIFT_DIR to point to the deeplift code (WITHIN the deeplift repo)");
sys.path.insert(0,scriptsDir);
#Make sure the directory is set to import the lab's version of keras
scriptsDir = os.environ.get("KERAS_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable KERAS_DIR");
sys.path.insert(0,scriptsDir)

import keras_conversion as kc

Using gpu device 0: GeForce GT 750M (CNMeM is disabled, CuDNN not available)
/Users/avantishrikumar/anaconda/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


Load the keras model

In [2]:
#Load the keras model, make sure you normalise the weights
#of the first convolutional layer to be mean-centered at each position.
model_weights = "apr11_deltaDeepLift_3task_2motif_deptask2_samplepwm_numSeq_20000_peakSize_200_DNNSimModel_20filt_15bp_plst50_200fc_do_epoch_42_modelWeights.h5"
model_yaml = "apr11_deltaDeepLift_3task_2motif_deptask2_samplepwm_numSeq_20000_peakSize_200_DNNSimModel_20filt_15bp_plst50_200fc_do_epoch_42_modelYaml.yaml"
reload(kc)
keras_model = kc.load_keras_model(model_weights, model_yaml, normalise_conv_for_one_hot_encoded_input=True)

Using Theano backend.


Load the data

In [3]:
scriptsDir = os.environ.get("UTIL_SCRIPTS_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable UTIL_SCRIPTS_DIR to point to the deeplift code");
sys.path.insert(0,scriptsDir);
from importDataPackage import importData
reload(importData)
trainData, validData, testData = importData.loadTrainTestValidFromYaml("yaml/features.yaml","yaml/labels.yaml", "yaml/splits.yaml")

Processed 10000 lines of apr10_deltaDeepLift_3task_2motif_deptask2_numSeq_20000_peakSize_200_TAL1_known1_GATA_known4_numSeq_20000_peakSize_200_freqMoMean_2_w_SPI1_known4_AP1_disc3_joint.fa.gz
Processed 20000 lines of apr10_deltaDeepLift_3task_2motif_deptask2_numSeq_20000_peakSize_200_TAL1_known1_GATA_known4_numSeq_20000_peakSize_200_freqMoMean_2_w_SPI1_known4_AP1_disc3_joint.fa.gz
('WARNING.', 'cls2_6001', 'was not found in train/test/valid splits')
This is the only time such a warning will be printed. Remaining such ids will be silently ignored
Processed 30000 lines of apr10_deltaDeepLift_3task_2motif_deptask2_numSeq_20000_peakSize_200_TAL1_known1_GATA_known4_numSeq_20000_peakSize_200_freqMoMean_2_w_SPI1_known4_AP1_disc3_joint.fa.gz
Processed 40000 lines of apr10_deltaDeepLift_3task_2motif_deptask2_numSeq_20000_peakSize_200_TAL1_known1_GATA_known4_numSeq_20000_peakSize_200_freqMoMean_2_w_SPI1_known4_AP1_disc3_joint.fa.gz
Processed 50000 lines of apr10_deltaDeepLift_3task_2motif_deptas

In [4]:
data = trainData

convert the keras sequential model into a deeplift sequential model

In [15]:
import blobs
reload(blobs)
deeplift_model = kc.convert_sequential_model(keras_model)

Compile the functions to compute the contributions and multipliers - the multipliers are analogous to the gradients

In [16]:
target_contribs_func = deeplift_model.get_target_contribs_func(input_layer_idx=0)
target_multipliers_func = deeplift_model.get_target_multipliers_func(input_layer_idx=0)

Compute the contributions for all 3 tasks and the multipliers for the third task

In [ ]:
contribs_datas = [np.array(target_contribs_func(task_idx=i, input_data_list=[data.X], batch_size=10, progress_update=10000))
                  for i in [0,1,2]]

multipliers_data_task2 = np.array(target_multipliers_func(task_idx=2, input_data_list=[data.X], batch_size=10, progress_update=10000))

Done 0
Done

Plot away! This relies on importing some plotting functions from the old DeepLIFT code base

In [ ]:
scriptsDir = os.environ.get("ENHANCER_SCRIPTS_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable ENHANCER_SCRIPTS_DIR to point to the enhancer_prediction_code repo");
sys.path.insert(0,scriptsDir+"/featureSelector/deepLIFFT");
from deepLIFTutils import makePngOfSequenceDeepLIFTScores

In [ ]:
idx = 19643 #coordinates of the GATA are 123-136

Plot the contribution scores for the three tasks

In [ ]:
for task in [0,1,2]:
    makePngOfSequenceDeepLIFTScores(contribs_datas[task][idx]
                                    , pngName="task"+str(task)+"_sequenceDLscores_weirdOne_"+str(idx)+".png")

Plot the multipliers for the third task

In [ ]:
makePngOfSequenceDeepLIFTScores(multipliers_data_task2[idx]
                                    , pngName="task"+str(task)+"_multipliers_weirdOne_"+str(idx)+".png")